In [1]:
project_folder = "/data/ATM/data_1/sfm/projects/EGU2"

In [2]:
import pandas as pd
import xml.etree.ElementTree as ET

def parse_gcp_xml(xml_file, type):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Create a list to store the data
    data = []

    if type == "image":
        # Iterate through each 'MesureAppuiFlottant1Im' in the XML
        for measure in root.findall('MesureAppuiFlottant1Im'):
            image_id = measure.find('NameIm').text
            # Iterate through each 'OneMesureAF1I' within the current 'MesureAppuiFlottant1Im'
            for pt in measure.findall('OneMesureAF1I'):
                gcp = pt.find('NamePt').text
                xy = pt.find('PtIm').text.split()
                x = int(xy[0])
                y = int(xy[1])

                # Append the extracted information to the data list
                data.append({'image_id': image_id, 'gcp': gcp, 'x': x, 'y': y})

        # Create a DataFrame
        df = pd.DataFrame(data, columns=['image_id', 'gcp', 'x', 'y'])

    elif type == "world":
        # Iterate through each 'OneAppuisDAF' in the XML
        for point in root.findall('OneAppuisDAF'):
            gcp = point.find('NamePt').text
            xy = point.find('Pt').text.split()
            x = float(xy[0])
            y = float(xy[1])
            incertitude = point.find('Incertitude').text.split()
            x_quality = float(incertitude[0])
            y_quality = float(incertitude[1])

            # Append the extracted information to the data list
            data.append({'gcp': gcp, 'x': x, 'y': y, 'x_quality': x_quality, 'y_quality': y_quality})

        # Create a DataFrame
        df = pd.DataFrame(data, columns=['gcp', 'x', 'y', 'x_quality', 'y_quality'])

    else:
        raise ValueError("Invalid type argument. Use 'image' or 'world'.")

    return df


In [10]:
import os
import geopandas as gpd
from shapely.geometry import Point

path_gcp_files_image = os.path.join(project_folder, "Measures-S2D.xml")
path_gcp_files_real = os.path.join(project_folder, "Measures.xml")

gcps_image = parse_gcp_xml(path_gcp_files_image, "image")
gcps_real = parse_gcp_xml(path_gcp_files_real, "world")

gcps_image.columns=['image_id', 'gcp', 'x_img', 'y_img']
gcps_real.columns=['gcp', 'x_real', 'y_real', 'x_quality', 'y_quality']

gcps = gcps_image.merge(gcps_real, on='gcp', how='inner')

# convert to geodataframe
geometry = [Point(x, y) for x, y in zip(gcps['x_real'], gcps['y_real'])]
geo_gcps = gpd.GeoDataFrame(gcps, geometry=geometry)
geo_gcps.set_crs(epsg=3031, inplace=True)

os.mkdir(os.path.join(project_folder, "debug"))

# save geo_cps as shapefile
geo_gcps.to_file(os.path.join(project_folder, "debug", "gcps.shp"))

print("geo_gcps saved at", os.path.join(project_folder, "debug", "gcps.shp"))

geo_gcps saved at /data/ATM/data_1/sfm/projects/EGU2/debug/gcps.shp
